In [31]:
import struct
import anndata as ad
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
from pathlib import Path
import pandas as pd
import numpy as np

def fgh(x):
    # mapp = {}
    l = []
    for i in list(x):
        if pd.isnull(i):
            l.append(np.nan)
            continue
        try:
            _ = float(i)
            l.append(float(i))
        except:
            print (i)
            if '<' in i: # i.strip() == "<8":
                l.append(float(i.split('<')[-1]))
                # mapp[i] = 7.99
            elif '>' in i:
                l.append(float(i.split('>')[-1]))
                # l.append(120.001)
                # mapp[i] = 120.001
    return l
    # return mapp
    
    
    
adata = ad.read("ml_generated_data/23_03_30/preprocessed_data/abc_adata.h5ad")
ml_generated_dir =  Path("block1_BaselineClassification/training/23_03_30/results_data")
ad_imaging_path = ml_generated_dir / "predictiveAnalysis/classification_detailed_metrics/phenotype_data/replication_ensemble_ad_imaging_probability_scores.csv"
pd_imaging_path = ml_generated_dir / "predictiveAnalysis/classification_detailed_metrics/phenotype_data/replication_ensemble_pd_imaging_probability_scores.csv"
ad2_imaging_data = pd.read_csv(ml_generated_dir / "predictiveAnalysis/classification_detailed_metrics/phenotype_data/ensemble_ad_imaging_probability_scores.csv")
pd2_imaging_data = pd.read_csv(ml_generated_dir / "predictiveAnalysis/classification_detailed_metrics/phenotype_data/ensemble_pd_imaging_probability_scores.csv")

ad_imaging_data = pd.read_csv(ad_imaging_path)
pd_imaging_data = pd.read_csv(pd_imaging_path)
ad_score_dictionary = dict(zip(list(ad_imaging_data['ID']), list(ad_imaging_data['rep_y_probas1'])))
ad_score_dictionary.update(dict(zip(list(ad2_imaging_data['ID']), list(ad2_imaging_data['y_probas1']))))
pd_score_dictionary = dict(zip(list(pd_imaging_data['ID']), list(pd_imaging_data['rep_y_probas1'])))
pd_score_dictionary.update(dict(zip(list(pd2_imaging_data['ID']), list(pd2_imaging_data['y_probas1']))))
from collections import defaultdict
all_study_diagnosis = adata.obs['study_diagnosis'].unique()
adata.obs['AD_imaging_score'] = adata.obs.index.map(lambda x: ad_score_dictionary.get(x, None))
adata.obs['PD_imaging_score'] = adata.obs.index.map(lambda x: pd_score_dictionary.get(x, None))
adata.obs['norm_AD_imaging_score'] = adata.obs['AD_imaging_score'].map(lambda x: np.log2(x / (1 - x)))
adata.obs['norm_PD_imaging_score'] = adata.obs['PD_imaging_score'].map(lambda x: np.log2(x / (1 - x)))
correlation_columns1 = ['norm_AD_imaging_score', 'norm_PD_imaging_score', ]
correlation_columns2 = ['mmse', 'mds_updrspartiii', 'moca', 'id_invicrot1_adas13', 'id_invicrot1_adas13', 
                        'id_invicrot1_tau', 'id_invicrot1_fdg', 'id_invicrot1_av45', 'id_invicrot1_abeta',
                        'id_csf_alpha_syn_csf alpha_synuclein', 'id_csf_abeta_42_abeta 1_42', 'id_dna_grs_grs',
                        
                       ]
correlation_columns2 = [
    'id_csf_p_tau181p_ptau', 'id_serum_nfl_nfl', 'id_csf_total_tau_ttau', 'id_csf_alpha_syn_csf alpha_synuclein', # 'id_csf_abeta_42_abeta 1_42'
]
correlation_columns1 = ['norm_AD_imaging_score', 'norm_PD_imaging_score', ]

correlation_columns2 = [
    'id_csf_alpha_syn_csf alpha_synuclein', 
    'id_serum_nfl_nfl',
    'id_csf_p_tau181p_ptau', 
    'id_serum_nfl_nfl', 
    'id_csf_total_tau_ttau',
    'id_subtype_cognitive_dimension',
    'id_subtype_motor_dimension',
    'id_subtype_sleep_dimension',
    'norm_PD_imaging_score'
]
    
results = defaultdict(list)
all_study_diagnosis = [
                'PPMI_PD',
                'PPMI_Control',
                'PPMI_SWEDD',
                'PPMI_GENPD',
                'PPMI_GENUN',
                'all'
                # 'PPMI_PD,PPMI_GENUN,PPMI_GENPD,PPMI_SWEDD,PPMI_Control'
                # 'ADNI_MCI',
                # 'ADNI_Control',
                # 'ADNI_Dementia',
]
for study_diagnosis in all_study_diagnosis:
    for col1 in correlation_columns1:
        for col2 in correlation_columns2:
            if study_diagnosis == 'all':
                T = adata.obs.copy()
            else:
                T = adata.obs[adata.obs['study_diagnosis'].isin(study_diagnosis.split(','))].copy()
            
            seed(1)
            # print (col1, col2, study_diagnosis)
            # print (T[col1].isna().sum(), T[col2].isna().sum())
            # print (len(T))
            
            if col2 == 'id_invicrot1_ptau' or col2 == 'id_invicrot1_abeta' or col2 == 'id_invicrot1_tau':
                T[col2] = fgh(T[col2].astype(str).map(lambda x: x[2:-1] if x is not x=='None' else np.nan))
            T = T.dropna(subset=[col1, col2])
            print (len(T))
            
            if len(T) <=0:
                corr, pvalue = None, None
            else:
                corr, pvalue = pearsonr(T[col1], T[col2])
            print (study_diagnosis, col1, col2, corr, pvalue, len(T))
            # sns.regplot(data=T, x=col1, y=col2)
            # plt.show()
            # results['feature'].append(col2)
            results['study_diagnosis'].append(study_diagnosis)
            results['column1'].append(col1)
            results['column2'].append(col2)
            results[f'correlation'].append(corr)
            results[f'pvalue'].append(pvalue)
            
Y = pd.DataFrame(results).copy()
Y['q'] = Y['study_diagnosis'] + '&'+ Y['column2']
Y['id'] = Y['column1']
Y['v'] = Y['correlation']
Y['pval'] = Y['pvalue']

new_results = pd.DataFrame(index=Y['id'].unique(), columns=Y['q'].unique())
for i in range(len(Y)):
    new_results.loc[Y.iloc[i]['id']][Y.iloc[i]['q']] = Y.iloc[i]['v']
    
annot_new_results = pd.DataFrame(index=Y['id'].unique(), columns=Y['q'].unique())
for i in range(len(Y)):
    if Y.iloc[i]['pval'] < 0.001:
        strr = '***'
    elif Y.iloc[i]['pval'] < 0.01:
        strr = '**'
    elif Y.iloc[i]['pval'] < 0.05:
        strr = '*'
    else:
        strr = ''
    annot_new_results.loc[Y.iloc[i]['id']][Y.iloc[i]['q']] = f"{round(Y.iloc[i]['v'], 2)}{strr}"

selected_columns_all = [
    ["ADNI_Control&id_invicrot1_adas13",
    "ADNI_MCI&id_invicrot1_adas13", 
    "ADNI_Dementia&id_invicrot1_adas13", ],
    
    ["ADNI_Control&mmse",
    "ADNI_MCI&mmse", 
    "ADNI_Dementia&mmse", ],
    
    ["ADNI_Control&moca", 
    "ADNI_MCI&moca", 
    "ADNI_Dementia&moca", ],
    
    ["PPMI_Control&mds_updrspartiii", 
    'PPMI_PD&mds_updrspartiii',], 

    ['PPMI_Control&moca',
    'PPMI_PD&moca',]
]

mapping = {
    "norm_AD_imaging_score": "AD imaging score",
    "norm_PD_imaging_score": "PD imaging score",
    "id_invicrot1_adas13": "ADAS-Cog-13",
    "mmse": "MMSE",
    "moca": "MOCA",
    "mds_updrspartiii": "MDS-UPDRS Part-III",
    "id_invicrot1_ptau": "PTau",
    "id_invicrot1_fdg": "FDG",
    "id_invicrot1_av45": "AV45",
    "id_invicrot1_abeta": "Abeta",
    "id_csf_alpha_syn_csf alpha_synuclein": "alpha-synuclein",  
}

662
PPMI_PD norm_AD_imaging_score id_csf_alpha_syn_csf alpha_synuclein -0.018873181186322974 0.6278747269981648 662
661
PPMI_PD norm_AD_imaging_score id_serum_nfl_nfl 0.37107630722999146 5.2321474750080753e-23 661
615
PPMI_PD norm_AD_imaging_score id_csf_p_tau181p_ptau 0.0860978147442442 0.032779158517407106 615
661
PPMI_PD norm_AD_imaging_score id_serum_nfl_nfl 0.37107630722999146 5.2321474750080753e-23 661
653
PPMI_PD norm_AD_imaging_score id_csf_total_tau_ttau 0.09816498176536913 0.012081575861821745 653
514
PPMI_PD norm_AD_imaging_score id_subtype_cognitive_dimension 0.10271250687638225 0.019851267064238173 514
514
PPMI_PD norm_AD_imaging_score id_subtype_motor_dimension 0.04753710237133521 0.2820502741534991 514
514
PPMI_PD norm_AD_imaging_score id_subtype_sleep_dimension -0.0032384338799382765 0.9416136924315368 514
684
PPMI_PD norm_AD_imaging_score norm_PD_imaging_score 0.161932944153182 2.0861332278858165e-05 684
662
PPMI_PD norm_PD_imaging_score id_csf_alpha_syn_csf alpha_synu

In [32]:
adata.obs['AD_imaging_score'].map(lambda x: np.log2(x / (1 - x))).mean()

-1.6188646859359461

In [33]:
adata.obs['AD_imaging_score'].map(lambda x: np.log(x / (1 - x))).mean()

-1.1221114927645746

In [34]:
adata.obs['norm_AD_imaging_score'] .mean()

-1.6188646859359461

In [35]:
adata

AnnData object with n_obs × n_vars = 11571 × 21
    obs: 'id_invicrot1_site', 'id_updrs4_np4wdysk', 'id_rem_slpinjur', 'study', 'id_csf_total sm_total sm', 'id_invicrot1_brainvolume', 'id_updrs3_np3postr', 'id_rem_rls', 'id_csf_c23 sm_c23 sm', 'id_updrs3_np3ktrmr', 'id_csf_c16 sm_c16 sm', 'id_epworth_ess2', 'id_updrs3_np3rign', 'id_updrs1pq_np1urin', 'id_vital_signs_syssup', 'id_invicrot1_fdg_bl', 'id_csf_c18 gl2_c18 gl2', 'id_csf_total cer_total cer', 'id_updrs4_np4dyski', 'id_invicrot1_mri_dti', 'id_csf_c20 glccer_c20 glccer', 'id_updrs2pq_np2turn', 'id_csf_c22 glccer_c22 glccer', 'id_invicrot1_moca', 'id_updrs3_np3brady', 'id_updrs3_pn3rigrl', 'id_invicrot1_icv_bl', 'id_rem_drmfight', 'id_invicrot1_mds_updrs_part3_remote_visit_on_or_off_updrdose_cat', 'id_csf_total gl2_total gl2', 'id_csf_c22 sm_c22 sm', 'id_epworth_ess7', 'id_rem_drmumv', 'id_vital_signs_diastnd', 'id_updrs3_np3rtall', 'id_updrs2pq_np2hygn', 'id_invicrot1_m', 'id_updrs3_np3rtaru', 'id_updrs2pq_np2frez', 'id_socio_e

In [36]:
[col for col in adata.obs.columns if 'subtype' in col]

['id_subtype_subtypes',
 'id_subtype_sleep_dimension',
 'id_subtype_cognitive_dimension',
 'id_subtype_motor_dimension']

In [37]:
required_columns = {
    'AD_imaging_score': "un_AD_score", 
    'PD_imaging_score': "un_PD_score",
    'norm_AD_imaging_score': "AD_score", 
    'norm_PD_imaging_score': "PD_score",
    'id_csf_alpha_syn_csf alpha_synuclein': "csf_alpha", 
    'id_serum_nfl_nfl': "serum_nfl", 
    'id_csf_p_tau181p_ptau': "csf_p_tau181", 
    'id_csf_total_tau_ttau': "csf_total_tau", 
    'id_subtype_cognitive_dimension': "subtype_cognitive", 
    'id_subtype_motor_dimension': "subtype_motor", 
    'id_subtype_sleep_dimension': "subtype_sleep", 
    'id_subtype_subtypes': "subtype",
    'moca': 'moca',
    'mmse': 'mmse', 
    'mds_updrspartiii': 'mds_updrspartiii', 
    'id_invicrot1_adas13': 'adas13',
    # 'id_invicrot1_tau': 'tau', 
    'id_invicrot1_fdg': 'fdg', 
    'id_invicrot1_av45': 'av45', 
    # 'id_invicrot1_abeta': 'abeta',
    # 'id_invicrot1_ptau': 'ptau'
}

In [38]:
H = adata.obs.copy()
s = H.groupby('subject_id').agg({'age': 'min'})
bage = dict(zip(list(s.index), list(s['age'])))
H['baseline_age'] = H['subject_id'].map(bage)
H['years_from_baseline'] = H['age'] - H['baseline_age']
# H = H[H['study_diagnosis']=='PPMI_PD']
# H['time'] = H['years_from_baseline'].values

In [39]:
mapping_gender = {'Male': 'M', 'Female': 'F', 'Unk': None}
H['gender'] = H['gender'].map(lambda x: mapping_gender.get(x, x))

In [40]:
more_cols = list(required_columns)
more_cols += ['subject_id', 'baseline_age', 'years_from_baseline', 'study_diagnosis', 'age', 'gender', 'id_subtype_subtypes']

In [41]:
for col in ['id_invicrot1_ptau', 'id_invicrot1_abeta', 'id_invicrot1_tau']:
    if col in H.columns:
        H[col] = fgh(H[col].astype(str).map(lambda x: x[2:-1] if x is not x=='None' else np.nan))

In [42]:
H[more_cols].rename(columns=required_columns).to_csv("streamlit_app/adni_ppmi_with_scores.csv", index=False)

In [43]:
K = H[more_cols].rename(columns=required_columns).copy()

In [44]:
K = K[~(K['study_diagnosis'].isin(['PPMI_None', 'ADNI_nan']))].copy()#  ['study_diagnosis'].value_counts()

In [45]:
s = K.groupby('subject_id').agg({'age': 'count'})
long_subjects = s[s['age']>1].index
not_longitudinal_data = K[~(K['subject_id'].isin(long_subjects))].copy()

In [46]:
longitudinal_data = K[K['subject_id'].isin(long_subjects)].copy()
crossectional_data = K.sort_values(by=['subject_id', 'age']).drop_duplicates(subset=['subject_id']).copy()
combined_data = K.copy()

In [47]:
combined_data.to_csv("streamlit_app/adni_ppmi_combined_data.csv", index=False)
crossectional_data.to_csv("streamlit_app/adni_ppmi_baseline_data.csv", index=False)
longitudinal_data.to_csv("streamlit_app/adni_ppmi_longitudinal_data.csv", index=False)

In [23]:
# last_visit_crossectional_data = K.sort_values(by=['subject_id', 'age']).drop_duplicates(subset=['subject_id'], keep='last').copy()
# last_visit_crossectional_data.to_csv("streamlit_app/adni_ppmi_last_visit_data.csv", index=False)

In [24]:
# longitudinal_data